In [15]:
import rasterio
from rasterio.merge import merge
from rasterio.mask import mask
import geopandas as gp
import numpy as np
import json
import os
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.io import MemoryFile
from shapely.geometry import mapping, box
import math

In [3]:
#paths
current = os.getcwd()
inp = os.path.dirname(current) + "/Data Outputs/"
tile_folder = os.path.dirname(os.path.dirname(current)) + "/Deforestation Raw Data/LandCoverEsri/"

In [4]:
gdf_CF = gp.read_file(inp + "Community_Forests_Zambia_V1.shp")

In [5]:
# dic of tifs
tile_paths_by_year = {
    2017: [os.path.join(tile_folder, "35L_20170101-20180101.tif"),
           os.path.join(tile_folder, "36L_20170101-20180101.tif")],
    2018: [os.path.join(tile_folder, "35L_20180101-20190101.tif"),
           os.path.join(tile_folder, "36L_20180101-20190101.tif")],
    2019: [os.path.join(tile_folder, "35L_20190101-20200101.tif"),
           os.path.join(tile_folder, "36L_20190101-20200101.tif")],
    2020: [os.path.join(tile_folder, "35L_20200101-20210101.tif"),
           os.path.join(tile_folder, "36L_20200101-20210101.tif")],
    2021: [os.path.join(tile_folder, "35L_20210101-20220101.tif"),
           os.path.join(tile_folder, "36L_20210101-20220101.tif")],
    2022: [os.path.join(tile_folder, "35L_20220101-20230101.tif"),
           os.path.join(tile_folder, "36L_20220101-20230101.tif")],
    2023: [os.path.join(tile_folder, "35L_20230101-20240101.tif"),
           os.path.join(tile_folder, "36L_20230101-20240101.tif")]
}


In [6]:
import os
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Define your destination CRS
dst_crs = 'EPSG:4326'

# Folder where your reprojected rasters will be saved
reprojected_folder = inp + "Reprojected"
os.makedirs(reprojected_folder, exist_ok=True)

# Loop through each year and each raster in your dictionary
for year, file_list in tile_paths_by_year.items():
    for tile_path in file_list:
        with rasterio.open(tile_path) as src:
            # Calculate transform, width, and height for the destination
            transform, width, height = calculate_default_transform(
                src.crs, dst_crs, src.width, src.height, *src.bounds)
            
            # Update metadata for destination
            kwargs = src.meta.copy()
            kwargs.update({
                'crs': dst_crs,
                'transform': transform,
                'width': width,
                'height': height
            })
            
            # Create an output filename; here we append _4326 before .tif
            base_name = os.path.basename(tile_path)
            dst_filename = os.path.join(reprojected_folder, base_name.replace('.tif', '_4326.tif'))
            
            with rasterio.open(dst_filename, 'w', **kwargs) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=dst_crs,
                        resampling=Resampling.nearest)
            
            print(f"Reprojected {tile_path} to {dst_filename}")

Reprojected /Users/vanessafigueroa/Documents/Personal projects/Deforestation/Deforestation Raw Data/LandCoverEsri/35L_20170101-20180101.tif to /Users/vanessafigueroa/Documents/Personal projects/Deforestation/DeforestationProject/Data Outputs/Reprojected/35L_20170101-20180101_4326.tif
Reprojected /Users/vanessafigueroa/Documents/Personal projects/Deforestation/Deforestation Raw Data/LandCoverEsri/36L_20170101-20180101.tif to /Users/vanessafigueroa/Documents/Personal projects/Deforestation/DeforestationProject/Data Outputs/Reprojected/36L_20170101-20180101_4326.tif
Reprojected /Users/vanessafigueroa/Documents/Personal projects/Deforestation/Deforestation Raw Data/LandCoverEsri/35L_20180101-20190101.tif to /Users/vanessafigueroa/Documents/Personal projects/Deforestation/DeforestationProject/Data Outputs/Reprojected/35L_20180101-20190101_4326.tif
Reprojected /Users/vanessafigueroa/Documents/Personal projects/Deforestation/Deforestation Raw Data/LandCoverEsri/36L_20180101-20190101.tif to /U

In [7]:
new_raster_folder = reprojected_folder + "/"

In [9]:
tile_paths_by_year_new = {
    2017: [os.path.join(new_raster_folder, "35L_20170101-20180101_4326.tif"),
           os.path.join(new_raster_folder, "36L_20170101-20180101_4326.tif")],
    2018: [os.path.join(new_raster_folder, "35L_20180101-20190101_4326.tif"),
           os.path.join(new_raster_folder, "36L_20180101-20190101_4326.tif")],
    2019: [os.path.join(new_raster_folder, "35L_20190101-20200101_4326.tif"),
           os.path.join(new_raster_folder, "36L_20190101-20200101_4326.tif")],
    2020: [os.path.join(new_raster_folder, "35L_20200101-20210101_4326.tif"),
           os.path.join(new_raster_folder, "36L_20200101-20210101_4326.tif")],
    2021: [os.path.join(new_raster_folder, "35L_20210101-20220101_4326.tif"),
           os.path.join(new_raster_folder, "36L_20210101-20220101_4326.tif")],
    2022: [os.path.join(new_raster_folder, "35L_20220101-20230101_4326.tif"),
           os.path.join(new_raster_folder, "36L_20220101-20230101_4326.tif")],
    2023: [os.path.join(new_raster_folder, "35L_20230101-20240101_4326.tif"),
           os.path.join(new_raster_folder, "36L_20230101-20240101_4326.tif")]
}

In [10]:
gdf_CF_rep = gdf_CF.to_crs(epsg=4326)

In [11]:
gdf_CF_rep

,name_origi,province,name_folde,area_ha,id,CF_desc,geometry
0,None,Muchinga Province,Sambaulye_CF_shapefiles,1565.777832,1,Sambaulye_CF_shapefiles,"POLYGON ((31.76043 -11.68503, 31.76395 -11.686..."
1,Kaloswe_CF_Original_Boundary,Muchinga Province,Kaloswe_CF_shapefiles,1047.517062,2,Kaloswe_CF_Original_Boundary,"POLYGON ((31.17501 -12.24452, 31.22262 -12.266..."
2,Mibobo CFA,Muchinga Province,Mibobo_CF_shapefiles,2484.858386,3,Mibobo CFA,"POLYGON ((30.52734 -12.32771, 30.56259 -12.360..."
3,Kakoko CF,Muchinga Province,Kakoko_CF_shapefiles,21431.252508,4,Kakoko CF,"POLYGON ((32.20133 -11.22931, 32.2027 -11.23, ..."
4,None,Muchinga Province,Lokomwila_CF_shapefiles,1887.019005,5,Lokomwila_CF_shapefiles,"POLYGON ((30.78484 -11.643, 30.79278 -11.63989..."
5,Chintasama_Community_Forest,Muchinga Province,Chintasama_CF_shapefiles,128.694483,6,Chintasama_Community_Forest,"POLYGON ((30.64096 -12.42662, 30.64643 -12.427..."
6,Chintasama_Community_Forest,Muchinga Province,Chintasama_CF_shapefiles,107.061293,7,Chintasama_Community_Forest,"POLYGON ((30.63193 -12.47547, 30.62709 -12.486..."
7,Chintasama_CF,Muchinga Province,Chintasama_CF_shapefiles,160.536332,8,Chintasama_CF,"POLYGON ((30.63687 -12.48847, 30.63334 -12.488..."
8,Chibulika CF,Muchinga Province,Chibulika_CF_shapefiles,59998.383889,9,Chibulika CF,"POLYGON ((32.56177 -11.07576, 32.5643 -11.3678..."
9,None,Muchinga Province,Kakoma_Samu_CF_shapefiles,251.700493,10,Kakoma_Samu_CF_shapefiles,"POLYGON ((33.41253 -10.4052, 33.41492 -10.4047..."


In [84]:
def calculate_feature_forest_area(feature_geom, tile_paths, forest_value=2):
    """
    For a given feature (polygon in EPSG:4326) and a list of raster tile paths (EPSG:4326),
    this function clips the raster to the feature, counts pixels equal to forest_value,
    and computes the forested area in hectares.
    
    Assumes each pixel is 10m x 10m because it comes from sentinel-2 data.
    """
    total_forest_area = 0.0
    # Constant pixel area: 10m x 10m = 100 m²
    pixel_area_ha = 100 / 10000.0  # equals 0.01 ha per pixel

    for fp in tile_paths:
        with rasterio.open(fp) as src:
            # Check if the feature intersects the raster bounds
            raster_bounds = box(*src.bounds)
            if not feature_geom.intersects(raster_bounds):
                continue
            try:
                # Clip the raster to the feature geometry
                clipped_data, clipped_transform = rasterio.mask.mask(src, [mapping(feature_geom)], crop=True)
            except Exception as e:
                # If clipping fails, skip this tile
                continue
            # Count pixels in the first band equal to the forest value
            forest_pixels = np.sum(clipped_data[0] == forest_value)
            total_forest_area += forest_pixels * pixel_area_ha
    return total_forest_area

results = []

for idx, feature in gdf_CF_rep.iterrows():
    geom = feature.geometry
    feature_area_ha = feature.area_ha

    # Retrieve attributes (adjust field names as needed)
    name_origi = feature.get('name_origi', 'Unknown')
    province   = feature.get('province', 'Unknown')
    name_folde = feature.get('name_folde', 'Unknown')
    CF_desc    = feature.get('CF_desc', name_folde)
    feature_id = feature.get('id', idx)
    
    # Calculate forested area for each year
    forest_area_by_year = {}
    for year, tile_paths in tile_paths_by_year_new.items():
        forest_area = calculate_feature_forest_area(geom, tile_paths, forest_value=2)
        forest_area_by_year[year] = forest_area
    
    # compute annual deforestation relative to the previous year.
    # (Positive values indicate forest loss relative to the previous year.)
    sorted_years = sorted(tile_paths_by_year_new.keys())
    previous_forest = None
    for year in sorted_years:
        current_forest = forest_area_by_year.get(year, 0.0)
        if previous_forest is None:
            # For the first year (e.g., 2017) no deforestation is computed.
            year_def = 0.0
        else:
            year_def = previous_forest - current_forest
        # Update previous forest for the next iteration
        previous_forest = current_forest
        
        # Compute percentage changes relative to the feature's total area
        year_perc_def = (year_def / feature_area_ha) if feature_area_ha != 0 else np.nan

        
        results.append({
            "name_origi": name_origi,
            "province": province,
            "name_folde": name_folde,
            "area_ha": feature_area_ha,
            "id": feature_id,
            "CF_desc": CF_desc,
            "for_area": current_forest,  # forested area in current year (ha)
            "year": year,
            "year_def_ha": year_def,      # deforestation relative to previous year (ha)
            "year_perc_def": year_perc_def  # annual deforestation (% of total area)
        })

# =============================================================================
# create dataframe!!!
# =============================================================================
results_df = pd.DataFrame(results)

In [85]:
results_df[results_df['id']==1]

,name_origi,province,name_folde,area_ha,id,CF_desc,for_area,year,year_def_ha,year_perc_def
0,None,Muchinga Province,Sambaulye_CF_shapefiles,1565.777832,1,Sambaulye_CF_shapefiles,1493.67,2017,0.00,0.000000
1,None,Muchinga Province,Sambaulye_CF_shapefiles,1565.777832,1,Sambaulye_CF_shapefiles,1520.90,2018,-27.23,-0.017391
2,None,Muchinga Province,Sambaulye_CF_shapefiles,1565.777832,1,Sambaulye_CF_shapefiles,1521.91,2019,-1.01,-0.000645
3,None,Muchinga Province,Sambaulye_CF_shapefiles,1565.777832,1,Sambaulye_CF_shapefiles,1527.06,2020,-5.15,-0.003289
4,None,Muchinga Province,Sambaulye_CF_shapefiles,1565.777832,1,Sambaulye_CF_shapefiles,1508.53,2021,18.53,0.011834
5,None,Muchinga Province,Sambaulye_CF_shapefiles,1565.777832,1,Sambaulye_CF_shapefiles,1505.70,2022,2.83,0.001807
6,None,Muchinga Province,Sambaulye_CF_shapefiles,1565.777832,1,Sambaulye_CF_shapefiles,1532.34,2023,-26.64,-0.017014


In [87]:
# Group by the feature id and sum the annual deforestation values
agg = results_df.groupby('id')['year_def_ha'].sum().reset_index().rename(columns={'year_def_ha': 'total_def'})

# Merge the total_def column back into the results DataFrame on the id field
results_df = results_df.merge(agg, on='id')

# Display the DataFrame with the new total_def column
results_df.head()

,name_origi,province,name_folde,area_ha,id,CF_desc,for_area,year,year_def_ha,year_perc_def,total_def
0,None,Muchinga Province,Sambaulye_CF_shapefiles,1565.777832,1,Sambaulye_CF_shapefiles,1493.67,2017,0.00,0.000000,-38.67
1,None,Muchinga Province,Sambaulye_CF_shapefiles,1565.777832,1,Sambaulye_CF_shapefiles,1520.90,2018,-27.23,-0.017391,-38.67
2,None,Muchinga Province,Sambaulye_CF_shapefiles,1565.777832,1,Sambaulye_CF_shapefiles,1521.91,2019,-1.01,-0.000645,-38.67
3,None,Muchinga Province,Sambaulye_CF_shapefiles,1565.777832,1,Sambaulye_CF_shapefiles,1527.06,2020,-5.15,-0.003289,-38.67
4,None,Muchinga Province,Sambaulye_CF_shapefiles,1565.777832,1,Sambaulye_CF_shapefiles,1508.53,2021,18.53,0.011834,-38.67


In [88]:
# plot

# interactive plot
import plotly.io as pio

pio.renderers.default = 'browser' 



In [90]:
# Identify a column with community forest names/IDs
id_col = "CF_desc" if "CF_desc" in results_df.columns else "id"
if id_col not in results_df.columns:
    results_df = results_df.reset_index().rename(columns={"CF_desc": "id"})
    id_col = "id"

# Create a line plot with Plotly Express
fig = px.line(
    results_df, 
    x="year", 
    y="year_def_ha", 
    color=id_col, 
    title="Yearly Deforestation by Community Forest",
    labels={"year": "Year", "year_def_ha": "Deforestation (ha)"}
)

# Move the legend to the right
fig.update_layout(
    legend=dict(
        x=1.05,  # position legend to the right
        y=1,
        orientation='v'
    )
)
fig.show()

In [ ]:
# save data 
results_df.to_excel(inp + 'Deforestation_and_Reforestation.xlsx', index=False)